In [2]:
import time
import os
import timm
import pandas as pd
import torch
import datasets

from torchvision import transforms

from dfic.common_functions.utility import count_model_parameters

/home/maciejzuziak/raid/DL_course/DFIC/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
DEVICE = 'cuda:0'

In [3]:
def train_epoch(
    train_loader,
    model,
    optimizer,
    loss_func
    ):    
    total_training_images = 0
    total_training_loss = 0
    total_training_correct = 0
    
    # Enumerating the data loader
    for i, data in enumerate(train_loader):
        inputs = data['image']
        labels = data['label']
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        # Removing previous gradient
        optimizer.zero_grad()
        # Using model inference
        outputs = model(inputs)
        # Calculating the loss
        loss = loss_func(outputs, labels)
        # Computing the gradients
        loss.backward()
        # Adjusting networking weights
        optimizer.step()
                
        # Training metrics
        total_training_images += labels.size(0)
        _, predicted = torch.max(outputs.detach(), 1)
        total_training_correct += (predicted == labels).sum().item()
        total_training_loss += loss.item()
    
    return {"total_training_images":total_training_images,
            "total_training_loss":total_training_loss,
            "total_training_correct":total_training_correct
            }

In [4]:
def validation_epoch(
    validation_loader,
    model,
    loss_func
    ):    
    total_validation_images = 0
    total_validation_loss = 0
    total_validation_correct = 0
    model.eval()
    
    # Enumerating the data loader
    with torch.no_grad():
        for i, data in enumerate(validation_loader):
            inputs = data['image']
            labels = data['label']
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)
            # Using model inference
            outputs = model(inputs)

            # Testing metrics
            total_validation_loss += loss_func(outputs, labels).item()
            total_validation_images += labels.size(0)
            _, predicted = torch.max(outputs.detach(), 1)
            total_validation_correct += (predicted == labels).sum().item()
        
    return {"total_validation_images":total_validation_images,
            "total_validation_loss":total_validation_loss,
            "total_validation_correct":total_validation_correct
            }

# Short-listed candidates
List of short-listed candidates due to their size and reported performance

In [9]:
models = ['resnet18', 'resnet26', 'resnet34', 'resnet50', 'mobilenetv3_small_050', 'mobilenetv3_small_075', 'mobilenetv3_small_100', 'convmixer_768_32']
model_evaluation = pd.DataFrame(index=models, columns=['number_of_parameters'])

for model in models:
    template = timm.create_model(model, num_classes=10)
    model_evaluation.loc[model, 'number_of_parameters'] = count_model_parameters(template)

model_evaluation.sort_values(by='number_of_parameters', inplace=True)
model_evaluation

,number_of_parameters
mobilenetv3_small_050,578474
mobilenetv3_small_075,1027122
mobilenetv3_small_100,1528106
resnet18,11181642
resnet26,13966666
convmixer_768_32,20348938
resnet34,21289802
resnet50,23528522


In [10]:
with open(os.path.join(os.getcwd(), 'model_selection', 'number_of_par_table.tex'), 'w') as file:
    file.write(model_evaluation.to_latex())

# Dataset

In [12]:
train_set = datasets.load_dataset('uoft-cs/cifar10', split='train')
out_set = datasets.load_dataset('uoft-cs/cifar10', split='test')
out_set = out_set.train_test_split(test_size=0.5)
test_set = out_set['test']
validation_set = out_set['train']

# train_set = train_set.with_format('torch', device='cuda:0')
# test_set = test_set.with_format('torch', device='cuda:0')
# validation_set = validation_set.with_format('torch', device='cuda:0')

# convert_tensor = transforms.ToTensor()
# train_set = train_set.map(lambda sample: {'image': convert_tensor(sample['image'])})
# train_set.set_format('pt', columns=['image'], output_all_columns=True)
# validation_set = validation_set.map(lambda sample: {'image': convert_tensor(sample['image'])})
# validation_set.set_format('pt', columns=['image'], output_all_columns=True)
# test_set = test_set.map(lambda sample: {'image': convert_tensor(sample['image'])})
# test_set.set_format('pt', columns=['image'], output_all_columns=True)

In [15]:
validation_set

Dataset({
    features: ['img', 'label'],
    num_rows: 5000
})

In [7]:
training_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=True)

In [11]:
final_test = pd.DataFrame(index=models, columns=['test loss', 'test accuracy'])
final_test

,test loss,test accuracy
resnet18,NaN,NaN
resnet26,NaN,NaN
resnet34,NaN,NaN
resnet50,NaN,NaN
mobilenetv3_small_050,NaN,NaN
mobilenetv3_small_075,NaN,NaN
mobilenetv3_small_100,NaN,NaN
convmixer_768_32,NaN,NaN


In [14]:
final_test.loc['resnet18', 'test loss']

nan

# Train-Validation-Test Evaluation of the Models

In [16]:
evaluation_frame = {
    model:
        {
            "training_loss": [],
            "validation_loss": [],
            "training_acc": [],
            'validation_acc': [],
        } for model in models
}

In [17]:
num_classes = 10
in_chans = 1
learning_rate = 0.0001

In [21]:
for model_name in models:
    model = timm.create_model(model_name, num_classes=num_classes, in_chans=in_chans, pretrained=False)
    model.to(DEVICE)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    loss_func = torch.nn.CrossEntropyLoss()
    print(f"Training model {model_name}")
    
    for epoch in range(1):
        ## TRAIN
        train_results = train_epoch(
            train_loader=training_loader,
            model=model,
            optimizer=optimizer,
            loss_func=loss_func
        )
        evaluation_frame[model_name]['training_loss'].append(train_results['total_training_loss'])
        evaluation_frame[model_name]['training_acc'].append(train_results['total_training_correct'] / train_results['total_training_images'])
        print(f"Epoch {epoch} training loss: {train_results['total_training_loss']:.3f}")
        print(f"Epoch {epoch} training accuracy: {train_results['total_training_correct'] / train_results['total_training_images']:.3f}")
        
        ## VALIDATE
        validation_results = validation_epoch(
            validation_loader=validation_loader,
            model=model,
            loss_func=loss_func
        )
        evaluation_frame[model_name]['validation_loss'].append(validation_results['total_validation_loss'])
        evaluation_frame[model_name]['validation_acc'].append(validation_results['total_validation_correct'] / validation_results['total_validation_images'])
        print(f"Epoch {epoch} validation loss: {validation_results['total_validation_loss']:.3f}")
        print(f"Epoch {epoch} validation accuracy: {validation_results['total_validation_correct'] / validation_results['total_validation_images']:.3f}")

    ## TEST
    test_results = validation_epoch(
        validation_loader=test_loader,
        model=model,
        loss_func=loss_func
    )
    print(f"Final test loss: {test_results['total_validation_loss']:.3f}")
    print(f"Final test accuracy: {test_results['total_validation_correct'] / test_results['total_validation_images']:.3f}")

    final_test.loc[model_name, 'test loss'] = test_results['total_validation_loss']
    final_test.loc[model_name, 'test accuracy'] = test_results['total_validation_correct'] / test_results['total_validation_images']


Training model resnet18
Epoch 0 training loss: 2600.029
Epoch 0 training accuracy: 0.733
Epoch 0 validation loss: 174.299
Epoch 0 validation accuracy: 0.849
Final test loss: 174.567
Final test accuracy: 0.851
Training model resnet26
Epoch 0 training loss: 2554.749
Epoch 0 training accuracy: 0.694
Epoch 0 validation loss: 184.644
Epoch 0 validation accuracy: 0.821
Final test loss: 183.685
Final test accuracy: 0.821
Training model resnet34
Epoch 0 training loss: 2200.941
Epoch 0 training accuracy: 0.810
Epoch 0 validation loss: 141.181
Epoch 0 validation accuracy: 0.889
Final test loss: 140.932
Final test accuracy: 0.888
Training model resnet50
Epoch 0 training loss: 1912.470
Epoch 0 training accuracy: 0.791
Epoch 0 validation loss: 98.222
Epoch 0 validation accuracy: 0.871
Final test loss: 98.094
Final test accuracy: 0.872
Training model mobilenetv3_small_050
Epoch 0 training loss: 4931.209
Epoch 0 training accuracy: 0.157
Epoch 0 validation loss: 410.175
Epoch 0 validation accuracy: 0.

In [33]:
for frame, results in evaluation_frame.items():
    with open(os.path.join(os.getcwd(), 'model_selection', f'{frame}.csv'), 'w') as file:
        file.write('epoch,training_loss,validation_loss,training_acc,validation_acc\n')
        for epoch in range(len(results['training_loss'])):
            file.write(f"{epoch},{results['training_loss'][epoch]},{results['validation_loss'][epoch]},{results['training_acc'][epoch]},{results['validation_acc'][epoch]}\n")

with open(os.path.join(os.getcwd(), 'model_selection', "final_test_results.csv"), 'w') as file:
    file.write(final_test.to_csv())